In [1]:
import numpy as np
import matplotlib.pyplot as plt

Defining the parameters

In [2]:
#massflowrates (kg/s)
massFlowOil = 0.03
massFlowWater = 0.1

#sp heat (J/kg-K)
spHeatOil = 2131
spHeatWater = 4178

#inlet temps (K)
tempInOil = 160 #433.15 
tempInWater = 35 #308.15

#dynamic viscosity (N-s/m^2)
viscOil = 3.25e-2 
viscWater = 725e-6

#Thermal Conductivity (W/m-K)
kOil = 0.138 
kWater = 0.625

#Prandtl Number
prOil = 501 
prWater = 4.85 

#hydrauilic diameter (m)
dOil = 30e-3 
dWaterO = 50e-3
dWaterI = 34e-3
dWater = dWaterO - dWaterI

#HE material properties Copper
price = 9.25 #£/kg
densityCu = 8900 #kg/m^3
kCu = 400 #W/m-K

In [3]:
cMin = spHeatOil*massFlowOil
cMax = spHeatWater*massFlowWater

cR = cMin/cMax

# aOil = (np.pi*(dOil**2))/4
# aWater = (np.pi*(dWaterO**2-dWaterI**2))/4


reOil = (4*massFlowOil*dOil)/(np.pi*dOil**2*viscOil)

reWater = (4*massFlowWater*dWater)/(np.pi*(dWaterO**2-dWaterI**2)*viscWater) 

In [4]:
approxL = np.linspace(0.1,2)
approxL = np.array([1,100])

def nuOiltest(al,adOil,areOil,aprOil):
    d = adOil/al        
    return (3.66+((0.0668*d*areOil*aprOil)/(1+(0.04*(pow(d*areOil*aprOil,2/3))))))

def nuWatercalc(re,pr,len,d):

    c = 1
    m = 2/3
    x = len/d
    f = pow((0.790*np.log(re))-1.64,-2)
    nufd = ((f/8)*(re-1000)*pr)/(1+((12.7*pow(f/8,1/2))*(pow(pr,2/3)-1)))

    Nu = nufd*(1+(c/pow(x,m)))
       
    return Nu, nufd


In [5]:
nuOil = 4.36

hOil = (nuOil * kOil)/dOil
tempOutWater = []
H_Overall = []

for L in approxL:
    
    x,y =  nuWatercalc(reWater,prWater,L,dWater)

    if L/dWater < 60:
        nuWater = x

    else:
         nuWater = y

    hWater = (nuWater * kWater)/dWater
    rWall = np.log(dWaterI/dOil)/((2*np.pi*kCu)*L)

    aOil = dOil*L 
    aWater = dWaterO*L #????

    overallH = (1/(hOil*aOil)) + rWall + (1/(hWater*aWater))

    # H_Overall.append(1/overallH)

    a = 1/overallH

    NTU = a/cMin
    b = NTU*(1 - cR)

    effectiveness = (1-np.exp(-b))/(1-(cR*np.exp(-b)))
    

    tempOutWater.append(((effectiveness*cMin*(tempInOil-tempInWater))/cMax) + tempInWater)
    


    


In [6]:
test1mOil = nuWatercalc(reWater,prWater,1,dWater)
test1mWater = nuOiltest(1,dWater,reWater,prWater) 

In [7]:
tempOutOil = tempInOil - ((cMin/cMax)*(tempOutWater[1] - tempInWater))